## Handling Imbalanced Datasets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
import itertools
from sklearn.externals import joblib

%matplotlib inline

## Helper methods

In [ ]:
def data_prepration(x): 
    x_features= x.iloc[:,x.columns != "Class"]
    x_labels=x.iloc[:,x.columns=="Class"]
    x_features_train,x_features_test,x_labels_train,x_labels_test = train_test_split(x_features,x_labels,test_size=0.3)
    return(x_features_train,x_features_test,x_labels_train,x_labels_test)


def printing_Kfold_scores(x_train_data,y_train_data):
    fold = KFold(len(y_train_data),5,shuffle=False) 

    # Different C parameters
    c_param_range = [0.01,0.1,1,1.5,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold,start=1):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        pass
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Load the data

In [ ]:
data = pd.read_csv("creditcard.csv")
data.head()

## Showing the ratio of classes

In [ ]:
count_of_classes = pd.value_counts(data['Class'], sort = True).sort_index()
count_of_classes.plot(kind = 'bar')
plt.title("Class Histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

# Showing ratio
print("Percentage of normal transactions: ", len(data[data.Class == 0])/len(data))
print("Percentage of fraud transactions: ", len(data[data.Class == 1])/len(data))
print("Total number of transactions in data: ", len(data))

## Data Preprocessing


In [ ]:
data['normalAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time','Amount'],axis=1)
data.head()

## Split data into training and test sets

In [ ]:
train_X, test_X, train_Y, test_Y = data_prepration(data)
columns = train_X.columns

## Use SMOTE to balance our dataset

In [ ]:
smote_algo = SMOTE(random_state=0)
smote_data_X,smote_data_Y = smote_algo.fit_sample(train_X, train_Y)
smote_data_X = pd.DataFrame(data=smote_data_X,columns=columns )
smote_data_Y= pd.DataFrame(data=smote_data_Y,columns=["Class"])
smote_data = smote_data_X
smote_data["Class"] = smote_data_Y["Class"]

In [ ]:
count_of_classes = pd.value_counts(smote_data_Y['Class'], sort = True).sort_index()
count_of_classes.plot(kind = 'bar')
plt.title("Class Histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

# Showing ratio
print("Percentage of normal transactions: ", len(smote_data_Y[smote_data_Y.Class == 0])/len(smote_data_Y))
print("Percentage of fraud transactions: ", len(smote_data_Y[smote_data_Y.Class == 1])/len(smote_data_Y))
print("Total number of transactions in balanced data: ", len(smote_data_Y))

## Split the balanced data into training and test sets

In [ ]:
smote_train_X, smote_test_X, smote_train_Y, smote_test_Y = data_prepration(smote_data)
columns = smote_train_X.columns

In [ ]:
best_c = printing_Kfold_scores(smote_train_X, smote_train_Y)

## Logisitic Regression Classifer on Imbalanced Data

In [ ]:
# Use the best C parameter to build the final model with the training
# dataset and predict the classes in the test dataset

logreg = LogisticRegression(C = best_c, penalty = 'l1')
fitted = logreg.fit(train_X, train_Y.values.ravel())
y_pred = logreg.predict(test_X.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(test_Y, y_pred)
np.set_printoptions(precision=2)

print("Recall metric in the test dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix')
plt.show()

# Plot ROC curves
y_pred_score = fitted.decision_function(test_X.values)
fpr, tpr, thresholds = roc_curve(test_Y.values.ravel(), y_pred_score)
roc_auc = auc(fpr,tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Logistic Regression Classifier on Balanced Data

In [ ]:
# Use the best C parameter to build the final model with the training
# dataset and predict the classes in the test dataset

logreg = LogisticRegression(C = best_c, penalty = 'l1')
fitted = logreg.fit(smote_train_X,smote_train_Y.values.ravel())
y_pred_smote = logreg.predict(smote_test_X.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(smote_test_Y, y_pred_smote)
np.set_printoptions(precision=2)

print("Recall metric in the test dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix')
plt.show()

# Plot ROC curves
y_pred_score = fitted.decision_function(smote_test_X.values)
fpr, tpr, thresholds = roc_curve(smote_test_Y.values.ravel(), y_pred_score)
roc_auc = auc(fpr,tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


## Persist our built model

In [ ]:
# Persist the model
joblib.dump(logreg, 'wiml_model.pkl')

# Persist the feature names
model_columns = list(test_X.columns)
joblib.dump(model_columns, 'model_columns.pkl') 